In [1]:
# bare minimum required imports
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Listing GPU and setting experimental memory growth to true for better performance
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [3]:
# constructing a simple sequential neural network
# attempts were made to do a transfer learning on vgg16 and resnet50, but that did not perform well
# maybe that would work for more generic detector
baseline_model = Sequential()
baseline_model.add(Conv2D(filters=32, kernel_size = (3,3), activation="relu", input_shape=(224, 224, 3)))
baseline_model.add(MaxPooling2D(pool_size=(2,2)))
baseline_model.add(BatchNormalization())
baseline_model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
baseline_model.add(MaxPooling2D(pool_size=(2,2)))
baseline_model.add(BatchNormalization())
baseline_model.add(Flatten())
baseline_model.add(BatchNormalization())
baseline_model.add(Dense(128,activation="relu"))
# dropout parameter had to be increased to 50% reduce overfitting
baseline_model.add(Dropout(0.5))
baseline_model.add(Dense(2, activation="softmax"))
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 111, 111, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                        

2022-03-23 11:34:59.394269: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 11:34:59.804255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4119 MB memory:  -> device: 0, name: Quadro P2000, pci bus id: 0000:65:00.0, compute capability: 6.1


In [4]:
folder = "./chest_xray"
train_path = folder + '/train'
model_histories = []
model_scores = []
models = []
split_coefs = [0.1,0.15,0.20,0.25,0.3,0.35]
# doing a n-fold training splitting the training dataset between test and train with ratios above
for split_coef in split_coefs:
    print("Training with split: ",str(split_coef))
    # creating a model from baseline model and compiling it
    model = baseline_model
    model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy']
    )
    # loading data in with some basic augmentation
    data_generator = ImageDataGenerator(preprocessing_function=tf.image.per_image_standardization,featurewise_center=False,
            samplewise_center=False,  
            featurewise_std_normalization=False,  
            samplewise_std_normalization=False,
            zca_whitening=False,  
            rotation_range=10, 
            zoom_range = 0.1, 
            width_shift_range=0.1,
            height_shift_range=0.1,  
            horizontal_flip=False,  
            vertical_flip=False,
            validation_split = split_coef)

    batch_size= 16
    # splitting the data between test and train
    train_generator = data_generator.flow_from_directory(directory=train_path, target_size=(224,224), classes=["normal","pneumonia"], batch_size=batch_size,subset='training')
    validation_generator = data_generator.flow_from_directory(directory=train_path, target_size=(224,224), classes=["normal","pneumonia"], batch_size=batch_size,subset='validation')
    # setting output file name
    filepath = "pneumonia-detection-model-via-loss-3-" + str(split_coef) + ".h5"
    # creating a model checkpoint callback for saving the best model for manual validation
    checkpoint = ModelCheckpoint(filepath=filepath, 
                                 monitor="val_loss",
                                 verbose=1, 
                                 save_best_only=True,
                                 mode="auto")
    # creatign a early stopping callback to minimise the training time, as having limited gpu resources
    early_stopping = EarlyStopping(
        monitor='val_loss', min_delta=0, patience=3, verbose=0,
        mode='auto', baseline=None, restore_best_weights=True
    )
    
    # training a model
    history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        validation_data = validation_generator, 
        validation_steps = validation_generator.samples // batch_size,
        epochs = 20,
        verbose = 1,
        callbacks=[checkpoint,early_stopping]
    )
    # saving best loss to model_scores list for determining best model after all training phases with all given split ratios
    best_model_loss = history.history['val_loss'][np.argmin(history.history['val_loss'])]
    model_scores.append(best_model_loss)
    # appending the saved model filename to list of models
    # it was done in this way rather than saving the actuall models to the list as that gave ram-related issues
    # models.append(model)
    models.append(filepath)


Training with split:  0.1
Found 2238 images belonging to 2 classes.
Found 248 images belonging to 2 classes.


/tmp/ipykernel_99773/4184927804.py:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20


2022-03-23 11:35:01.603226: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302


139/139 [==============================] - ETA: 0s - loss: 2.5789 - accuracy: 0.8479
Epoch 1: val_loss improved from inf to 3.86631, saving model to pneumonia-detection-model-via-loss-3-0.1.h5
139/139 [==============================] - 46s 321ms/step - loss: 2.5789 - accuracy: 0.8479 - val_loss: 3.8663 - val_accuracy: 0.5500
Epoch 2/20
139/139 [==============================] - ETA: 0s - loss: 2.3200 - accuracy: 0.8722
Epoch 2: val_loss did not improve from 3.86631
139/139 [==============================] - 42s 304ms/step - loss: 2.3200 - accuracy: 0.8722 - val_loss: 11.6701 - val_accuracy: 0.5208
Epoch 3/20
139/139 [==============================] - ETA: 0s - loss: 2.0366 - accuracy: 0.8987
Epoch 3: val_loss improved from 3.86631 to 1.88520, saving model to pneumonia-detection-model-via-loss-3-0.1.h5
139/139 [==============================] - 45s 322ms/step - loss: 2.0366 - accuracy: 0.8987 - val_loss: 1.8852 - val_accuracy: 0.8708
Epoch 4/20
139/139 [==============================] -

In [8]:
print(np.argmin(model_scores))
# Determining the best split coeficient via minimal loss value saved
best_split = split_coefs[np.argmin(model_scores)]
# Determining the best model name via minimal loss value saved
best_model = models[np.argmin(model_scores)]
# Printing out the results
print("Best model is: ", models[np.argmin(model_scores)])
print("Best split is: ", split_coefs[np.argmin(model_scores)])

4
Best model is:  pneumonia-detection-model-via-loss-3-0.3.h5
Best split is:  0.3


In [6]:
print(model_scores)

[0.9418102502822876, 0.7152588963508606, 0.6592831611633301, 0.5884536504745483, 0.5273534059524536, 0.536533534526825]
